<a href="https://colab.research.google.com/github/enesergen/Fashion_Models/blob/main/Model_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
zip_ref=zipfile.ZipFile("/content/drive/MyDrive/output.zip")
zip_ref.extractall()
zip_ref.close()

In [ ]:
import shutil
import os


In [ ]:
shutil.rmtree("/content/output/val/Handbags")
shutil.rmtree("/content/output/test/Handbags")
shutil.rmtree("/content/output/train/Handbags")

In [ ]:
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# must be controlled whether the path is correct or not
train_dir="./output/train/"
test_dir="./output/test/"
val_dir="./output/val/"

data_dir=pathlib.Path(train_dir)
class_names=np.array(sorted([item.name for item in data_dir.glob("*")]))
print(class_names)

train_datagen_augmented=ImageDataGenerator(rescale=1/255.,
                                           rotation_range=20,
                                           width_shift_range=0.2,
                                           height_shift_range=0.2,
                                           zoom_range=0.3,
                                           horizontal_flip=True)

train_data_augmented=train_datagen_augmented.flow_from_directory(train_dir,
                                                                 target_size=(224,224),
                                                                 batch_size=32,
                                                                 class_mode="categorical")
val_datagen=ImageDataGenerator(rescale=1/255.)
test_datagen=ImageDataGenerator(rescale=1/255.)
val_data=val_datagen.flow_from_directory(val_dir,
                                             target_size=(224,224),
                                             batch_size=32,
                                             class_mode="categorical")
test_data=test_datagen.flow_from_directory(test_dir,
                                             target_size=(224,224),
                                             batch_size=32,
                                             class_mode="categorical")

tf.random.set_seed(42)

base_model = VGG19(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

for layer in base_model.layers:
  layer.trainable = False

model_7=Sequential([
    base_model,
    Flatten(),
    Dense(12, activation="softmax")
])

model_7.compile(loss="categorical_crossentropy",
                optimizer=Adam(),
                metrics=["accuracy"])

history_7=model_7.fit(train_data_augmented,
                      epochs=30,
                      steps_per_epoch=len(train_data_augmented),
                      validation_data=val_data,
                      validation_steps=len(val_data))

model_7.evaluate(test_data)

# Plot the validation and training data separately
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

plot_loss_curves(history_7)

model_7.save("Model_7")

model_7.save("Model_7_h5.h5")

KeyboardInterrupt: ignored